## Import

In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
import torchsummary
import torch.nn.functional as F

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
!nvidia-smi

Thu Jul 27 21:11:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.75       Driver Version: 462.75       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 306... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8    13W /  N/A |    121MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%cd open (2)

C:\Users\user\Desktop\deep learning\Building_Segmentation\open (2)


In [4]:
# SEED 고정
CFG = {
    "epochs": 30,
    "learning_rate": 5e-5,
    "batch_size": 4,
    "seed": 42
}

def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG["seed"])

### Data Augmentation 전에 우선 train, valid, test split
- 5500: train, 1000: validation, 640: test
- test_set을 통해 loss값 확인 및 DICE SCORE 산식 적용해서 확인할 예정
- train에 대해서 직접 augmentation을 적용 시키고자 했으나 albumentaion으로 진행하는 것으로 변경했다.
- 따로 이 부분은 활용하지 않을 예정.

In [5]:
train = pd.read_csv("train.csv")
train_set = train[:6140]
valid_set = train[6140:]
train_set.to_csv("trainset.csv")
valid_set.to_csv("validset.csv")
# # # 2번째 테스트
train2 = pd.read_csv("train.csv")
train_set = train2[1000:]
valid_set = train2[:1000]
train_set.to_csv("trainset2.csv")
valid_set.to_csv("validset2.csv")

## Utils

In [6]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Custom Dataset

In [7]:
class SatelliteDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False, num = None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer
        self.num = num
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.num == 0:
            # csv파일을 불러와 column을 잘 보면 한 column은 파일의 저장 경로이다.
            img_path = self.data.iloc[idx, 2]
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)        
            if self.infer:
                if self.transform:
                    image = self.transform(image=image)['image']
                return image

            mask_rle = self.data.iloc[idx, 3]
            mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

            if self.transform:
                augmented = self.transform(image=image, mask=mask)
                image = augmented['image']
                mask = augmented['mask']

            return image, mask
        elif self.num == 1:
            # csv파일을 불러와 column을 잘 보면 한 column은 파일의 저장 경로이다.
            img_path = self.data.iloc[idx, 2]
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)        
            if self.infer:
                if self.transform:
                    image = self.transform(image=image)['image']
                return image

            mask_rle = self.data.iloc[idx, 3]
            mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

            if self.transform:
                augmented = self.transform(image=image, mask=mask)
                image = augmented['image']
                mask = augmented['mask']

            return image, mask
        else:
            img_path = self.data.iloc[idx, 1]
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)        
            if self.infer:
                if self.transform:
                    image = self.transform(image=image)['image']
                return image

            mask_rle = self.data.iloc[idx, 2]
            mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

            if self.transform:
                augmented = self.transform(image=image, mask=mask)
                image = augmented['image']
                mask = augmented['mask']

            return image, mask

## Data Loader
- 괜찮은 transform 조합을 찾는다.
- transform 조합을 통해 더 나은 결과를 낸다.

In [8]:
transform_train_base = A.Compose(
    [   
        A.augmentations.crops.transforms.CropNonEmptyMaskIfExists(height = 224, width = 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]
)
# 기본적인 augmentation: random crop, horizontal, vertical, rotate, blur, normalize
transform_train_aug = A.Compose(
    [   
        A.augmentations.crops.transforms.CropNonEmptyMaskIfExists(height = 224, width = 224),
        A.OneOf([A.HorizontalFlip(p = 0.5), A.VerticalFlip(p = 0.5)], p = 0.8),
        A.Rotate(limit = 180, p = 0.5, border_mode = cv2.BORDER_REPLICATE),
        A.GaussianBlur(blur_limit = (3, 7), always_apply = False, p = 0.5),
        A.GridDropout(ratio = 0.2, random_offset = True, holes_number_x=4,
                      holes_number_y= 4,p = 0.8),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]
)

# transform_train_aug2 = A.Compose(
#     [   
#         A.CenterCrop(224, 224),
#         A.OneOf([A.HorizontalFlip(p = 0.5), A.VerticalFlip(p = 0.5)]),
# #         A.Rotate(limit = 180, p = 0.5, border_mode = cv2.BORDER_REPLICATE),
#         A.OneOf([A.GaussianBlur(blur_limit = (3, 7), always_apply = False, p = 0.5), A.GaussNoise()]),
#         A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         ToTensorV2()
#     ]
# )

transform_other = A.Compose(
    [   
#         A.augmentations.crops.transforms.CropNonEmptyMaskIfExists(height = 224, width = 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]
)


transform_other1 = A.Compose(
    [   
        A.RandomCrop(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)
# transform_other2 = A.Compose(
#     [   
#         A.RandomCrop(224, 224),
#         A.Normalize(),
#         ToTensorV2()
#     ]
# )
# transform_version2 = A.Compose(
#     [   
#         A.CenterCrop(224, 224),
# #         A.HorizontalFlip(p = 0.5),
#         A.Rotate(limit = 180, p = 0.8, border_mode = cv2.BORDER_REPLICATE),
# #         A.VerticalFlip(p = 0.5),
# #         A.HueSaturationValue(hue_shift_limit = 20, sat_shift_limit = 30, val_shift_limit = 20, p = 0.5),
#         A.GaussianBlur(blur_limit = (3, 7), always_apply = False, p = 0.5),
#         A.Normalize(),
#         ToTensorV2()
#     ]
# )


# pretrained 된 것 더 이어서 학습 시키는 용도
# 이럴 경우는 주의해서 load 시킬 것
# crop_transform = A.Compose(
#     [   
#         A.RandomCrop(224, 224),
#         A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
#         A.RandomRotate90(p=0.5),
#         A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
#         A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
#         A.CLAHE(clip_limit=4.0, tile_grid_size=(8,8), p=0.5),
#         A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         ToTensorV2()
#     ]
# )
train_dataset_base = SatelliteDataset(csv_file='./trainset.csv', transform = transform_train_base, num = 0)
train_dataset_aug = SatelliteDataset(csv_file = "./trainset.csv", transform = transform_train_aug, num = 0)
# train_set = SatelliteDataset(csv_file = "./train.csv", transform = crop_transform, num = 0)
# train_dataset_aug2 = SatelliteDataset(csv_file = "./trainset.csv", transform = transform_train_aug2, num = 0)
train_dataset = train_dataset_base + train_dataset_aug
train_loader = DataLoader(train_dataset, batch_size = CFG["batch_size"], shuffle=True, num_workers=0)
valid_dataset = SatelliteDataset(csv_file="./validset.csv", transform = transform_other1, num = 1)
valid_dataset = valid_dataset
valid_loader = DataLoader(valid_dataset, batch_size = CFG["batch_size"], shuffle = False, num_workers = 0)
# test_dataset = SatelliteDataset(csv_file = "./testset.csv", transform = transform_other, num = 1)
# test_loader = DataLoader(test_dataset, batch_size = CFG["batch_size"], shuffle = False, num_workers = 0)
# validation 추가
# train_dataset = SatelliteDataset(csv_file = "./train.csv", transform = transform_train_base, num = 0)

## Define Model
- CVPR 2020에 발표된 Eff-UNet: A Novel Architecture for Semantic Segmentation in Unstructured Environment 활용
- 총 7개의 MBConv Block을 이용
- 논문에서 제시한 block을 통과한 후의 channel와 pretrained를 위한 channel의 수의 차이가 존재
- encoder 부분은 이를 참고해 EfficientNet_v2_m을 적용.
- decoder 부분은 U_Net의 decoder 활용.
- decoder 부분을 변형 시킬 예정
    - 특히 단순히 U-Net이 아닌 U-Net++, U-Net3+의 아이디어도 참고할 예정


※ 추가적으로 customized loss, DICE Score 확인해가며 진행할 예정

In [9]:
from efficientnet_pytorch import EfficientNet


# U-Net의 기본 구성 요소인 Double Convolution Block을 정의합니다.
# 기본 model에는 BatchNormalization이 존재하지 않으므로 따로 정의했다.
# U_Net Decoder의 마지막 부분 참고하여 Conv2d(32, 1)로 변형했다.
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

# EFFICIENTNET-V2 ENCODER + U_NET DECODER
# pretrained_task를 이용할 예정.

class EFFv2_UNet(nn.Module):
    
    def __init__(self):
        super(EFFv2_UNet, self).__init__()
        
        # EfficientNet_V2_L + U_Net decoder
        self.backbone = models.efficientnet_v2_m(pretrained = True, weights='EfficientNet_V2_M_Weights.DEFAULT')
        # block 설정
        self.feature1 = self.backbone.features[0]
        self.feature2 = self.backbone.features[1]
        self.feature3 = self.backbone.features[2]
        self.feature4 = self.backbone.features[3]
        self.feature5 = self.backbone.features[4]
        self.feature6 = self.backbone.features[5]
        self.feature7 = self.backbone.features[6]

        # upsample은 6회 진행.
        # 해당 코드는 이미지나 tensor의 크기를 2배로 늘려주는 역할을 한다.
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        
        # concatenate: 512 channels, 256 channels
        # 7 -> 14
        self.dconv_up6 = double_conv(304, 512)
        self.dconv_up5 = double_conv(512 + 176, 256)
        # concatenate: 256 channels, 128 channels
        # 14 -> 28
        self.dconv_up4 = double_conv(256 + 160, 128)
        # concatenate: 128 channels, 64 channels
        # 28 -> 56
        self.dconv_up3 = double_conv(128 + 80, 64)
        # 56 -> 112
        self.dconv_up2 = double_conv(64 + 48, 64)
        # 112 -> 224
        self.dconv_up1 = double_conv(64 + 24, 32)
        # last process
        self.conv_last = nn.Conv2d(32, 1, 1)

    def forward(self, x):
        # efficientnet의 모든 block을 통과한다.
        # convolution 저장(추후에 concatenate)
        conv1 = self.feature1(x)
        x = self.feature1(x)
        conv2 = self.feature2(x)
        x = self.feature2(x)
        conv3 = self.feature3(x)
        x = self.feature3(x)
        conv4 = self.feature4(x)
        x = self.feature4(x)
        conv5 = self.feature5(x)
        x = self.feature5(x)
        conv6 = self.feature6(x)
        x = self.feature6(x)
        # 이 자체가 upsampling + conv2d를 지난다.
        x = self.feature7(x)
        
        # 논문에 따르면 우선 마지막 block의 결과는 UpSampling 후 Conv2D를 거친다
        # 논문과는 약간 결이 다르지만 일단 전의 정보를 최대한 반영해서 학습을 진행하도록 했다.
        x = self.dconv_up6(x)
        # size가 14로 된다.
        x = self.upsample(x)
        # conv6의 경우는 concatenate된다
        # x의 차원은 512, conv6는 112
        x = torch.cat([x, conv6], dim=1)
        # 여기까지 하면 size는 14, channel 수는 512 + 112
        x = self.dconv_up5(x)
        # 이를 통과하면 x는 256이 된다
        x = torch.cat([x, conv5], dim=1)
        # 28
        x = self.upsample(x)
        x = self.dconv_up4(x)
        x = torch.cat([x, conv4], dim=1)
        # 56
        x = self.upsample(x)
        x = self.dconv_up3(x)
        x = torch.cat([x, conv3], dim=1)
        # 112
        x = self.upsample(x)
        x = self.dconv_up2(x)        
        x = torch.cat([x, conv2], dim=1)
        # 112
        x = self.dconv_up1(x)
        x = self.upsample(x) 

        out = self.conv_last(x)
        # 따로 sigmoid를 통과하지는 않는다.

        return out

In [10]:
def validation(model, valid_loader, criterion, device):
    model.eval()
    val_loss = 0
    # gradient 추적 x
    with torch.no_grad():
        for X, y in tqdm(valid_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            out = model(X)
            loss = criterion(out, y.unsqueeze(1))
            val_loss += loss.item()
    val_loss = val_loss / len(valid_loader)
    return val_loss

## Model Train

In [20]:
# model 초기화
# model = EFFv2_UNet().to(device)
model = EFFv2_UNet()
model.load_state_dict(torch.load("./0.2557478419079834_checkpoint.pt"))


def train(model, optimizer, criterion, train_loader, valid_loader, scheduler, device):
    model.to(device)
    # training loop
    best_loss = 999999
    best_model = None
    x_range = range(1, CFG["epochs"]+1)
    # for early_stopping
    PATIENCE = 7
    cnt = 0
    for epoch in range(CFG["epochs"]):
        model.train()
        epoch_loss = 0
        for images, masks in tqdm(train_loader):
            images = images.float().to(device)
            masks = masks.float().to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks.unsqueeze(1))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        valid_loss = validation(model, valid_loader, criterion, device)
        
        if scheduler is not None:
            scheduler.step(valid_loss)
        
        if best_loss > valid_loss:
            best_loss = valid_loss
            # validation 기준으로 최적의 모델 저장
            cnt = 0
            best_model = model
            print("validation_loss의 minimum epoch 시기 {}, val_loss값 {}".format(epoch+1, valid_loss))
        else:
            cnt += 1
        if cnt > PATIENCE:
            print("Validation loss renewal stopped")
            return best_model
        
        print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(train_loader)}')

    return best_model

In [21]:
# dice score와 cross entropy loss를 결합한다. 
class DiceCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(DiceCrossEntropyLoss, self).__init__()
        # Binary Cross Entropy logloss 모듈
        self.BCELoss = torch.nn.BCEWithLogitsLoss()
    def forward(self, outputs, targets, smooth=1):
        # binary cross entropy logloss 계산
        BCELOSS = self.BCELoss(outputs, targets)
        # dice score 계산하는 부분
        # torch.sigmoid 또는 torch.nn.function의 sigmoid를 활용 가능하다.
        inputs = torch.sigmoid(outputs)
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        # 기본적인 dice score 계산(smooth factor는 분모가 0이 되는 것을 방지)
        dice = (2.*intersection + smooth) / (inputs.sum() + targets.sum() + smooth)
        # dice의 경우는 1에 근접할 수록 좋은 score이다. 따라서 loss로 활용하고자 한다면 반대로 진행
        # loss가 0으로 수렴할 수도 있어서 보통 변수 결합 하여 모든 것을 고려할 때는 곱하지만 더하는 방향으로 진행했습니다.
        combined_loss = (1-dice) + BCELOSS
        
        return combined_loss

In [22]:
# 기준은 best validation model을 기준으로 설정.
# loss function과 optimizer 정의
# train_loss: 0.27아래, validation loss: 0.25 아래로 나올 수 있도록 계속해서 실험할 예정. validation 기준으로 dice score 예상: 74점
criterion = DiceCrossEntropyLoss()
# optimizer = torch.optim.AdamW(new_model.parameters(),lr=CFG["learning_rate"])
optimizer = torch.optim.AdamW(model.parameters(), lr = CFG["learning_rate"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = "min", factor = 0.5, patience = 3, threshold_mode = 'abs', min_lr = 1e-5, verbose = True)
best_model = train(model, optimizer, criterion, train_loader, valid_loader, scheduler, device)

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:44<00:00,  5.61it/s]


validation_loss의 minimum epoch 시기 1, val_loss값 0.24833990561962127
Epoch 1, Loss: 0.2928720819469578


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:38<00:00,  6.52it/s]


validation_loss의 minimum epoch 시기 2, val_loss값 0.24679794321954251
Epoch 2, Loss: 0.2781615990393034


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:37<00:00,  6.60it/s]


...

Epoch 22, Loss: 0.2563194066650122


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:36<00:00,  6.78it/s]


Epoch 23, Loss: 0.252709308669505


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:36<00:00,  6.82it/s]

Epoch 00024: reducing learning rate of group 0 to 1.2500e-05.
Validation loss renewal stopped


In [24]:
# pt file을 save
torch.save(best_model, "./best_model_efficientunetv2m_4batch_augmentation_7_28_dicescore.pth")
torch.save(model, "./last_model_7_28.pth")

In [ ]:
!nvidia-smi

## Inference

In [25]:
test_dataset = SatelliteDataset(csv_file='./test.csv', transform=transform_other1, infer=True, num = 2)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=0)

In [29]:
with torch.no_grad():
    best_model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)
        
        outputs = best_model(images)
        masks = torch.sigmoid(outputs).cpu().numpy()
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.35).astype(np.uint8) # Threshold = 0.35
        
        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)

100%|████████████████████████████████████████████████████████████████████████████| 15160/15160 [15:32<00:00, 16.25it/s]


## Submission

In [30]:
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result

In [31]:
submit.to_csv('./submit.csv', index=False)

### try 1 (7/4)
- 30 epochs
- 데이터를 4500장, 1500장, 1140장으로 나눠서 진행
- train에 해당하는 4500장 중 1000장에 대해서 noise를 추가해서 진행
- Compose의 경우는 단순히 224x224로 Resize 진행
- random seed는 고정하지 않았음
- pt file을 저장.
### try 2 (7/5)
- try 1에서의 pt file을 불러와서 진행
- train은 앞의 5000장, validation은 1500장으로 진행
- random seed 값은 42로 고정
- Compose의 경우는 단순히 Albumentation의 RandomCrop으로 진행
- 20 epoch 추가 수행
- validation loss는 0.07, train loss는 0.08
- pt file을 저장
### try 3(7/6)
- 50 epochs
- train set은 5000장, noise augmentation은 500장, flip, horizontal flip을 train에 적용
- 1500장의 validation set
- validation loss는 0.07, train

※ U-Net 역시 decoder 부분에 대한 pretrained task weight를 넣어 진행해 볼 것

### final
- 최종적으로 얼마나 잘 test 데이터에 대해서 체크를 하는 지 확인해 볼 예정이다.